安裝python3.6

In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.6

替換python版本

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 3
!python --version
!sudo apt-get install python3.6-distutils
!wget https://bootstrap.pypa.io/pip/3.6/get-pip.py
!python get-pip.py
# upgrade pip
!sudo apt install python3-pip
!python -m pip install --upgrade pip

修正Tersonflow與Keras版本

In [ ]:
!pip uninstall keras
!pip install tensorflow==2.2.0
!pip install Keras==2.3.1

匯入雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/My\ Drive/Colab\ Notebooks/imbd2019
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/imbd2019/')

載入配置

In [1]:
class _config:
    # train 資料夾位置
    Train_Path = './train/'
    # Train_Path = '/content/drive/My Drive/Colab Notebooks/imbd2019/train/'
    # test 資料夾位置
    Test_Path = './test/'
    # Test_Path = '/content/drive/My Drive/Colab Notebooks/imbd2019/test/'
    # img size
    imgrow = 164
    imgcol = 4
    # PTC 類別定義
    PTC_class = {'G11': 0, 'G15': 1, 'G17': 2, 'G19': 3,
                 'G32': 4, 'G34': 5, 'G48': 6, 'G49': 7}
    test_y = {'1.txt': 'G11', '2.txt': 'G11',
              '3.txt': 'G15', '4.txt': 'G15', '5.txt': 'G15', '6.txt': 'G15', '7.txt': 'G15', '8.txt': 'G15',
              '9.txt': 'G17', '10.txt': 'G17',
              '11.txt': 'G19', '12.txt': 'G19',
              '13.txt': 'G32', '14.txt': 'G32', '15.txt': 'G32', '16.txt': 'G32', '17.txt': 'G32', '18.txt': 'G32',
              '19.txt': 'G34', '20.txt': 'G34', '21.txt': 'G34', '22.txt': 'G34', '23.txt': 'G34', '24.txt': 'G34',
              '25.txt': 'G48', '26.txt': 'G48', '27.txt': 'G48', '28.txt': 'G48', '29.txt': 'G48', '30.txt': 'G48',
              '31.txt': 'G49', '32.txt': 'G49', '33.txt': 'G49', '34.txt': 'G49', '35.txt': 'G49', '36.txt': 'G49',
              }
    Batch_size = 32
    epoch = 70
    class_num = 8


載入檔案

In [2]:
import sys
from os import walk
import numpy as np
import cv2
#from config import _config

class _file:

    def __init__(self):
        self.Set = _config()
        pass

    def ReadData_TxT(self, Path):
        AllPTCNp = np.zeros((1, self.Set.imgcol))
        AllDataList = list()
        labellsit = list()
        PTClabelList = list()
        namelist = list()
        # 讀取資料夾內檔案
        for root, dirs, file_all in walk(Path):

            for _file in dirs:
                # 讀取各類別資料夾(G11、G15、G17、G19、G32、G34、G48、G49)
                for root, dirs, file_all in walk(Path+_file):
                    # 讀取各類別資料夾內的TXT
                    for name in file_all:
                        # print(name)
                        namelist.append(name)
                        Datalist = list()
                        SingleTxTNp = np.zeros((1, 4))
                        PTCList = list()
                        StartSaveData = False

                        # 讀取TXT
                        f = open(Path+_file+'/'+name, 'r', encoding='big5')
                        data = f.readlines()
                        for i in data:
                            Datalist.append(i)
                        f.close()

                        # 讀取TXT內容後，淨化資料內容
                        for i in Datalist:
                            # 判斷此TXT類別
                            if i.find('G') > -1:
                                label = i[i.find('G'):i.find('-')]

                                labellsit.append(label)
                            # 經過Deg.F 確認開始讀取 PTC
                            if StartSaveData:
                                PTCList.append(i)

                                # 若讀到換行符號，row方向往下堆疊
                                if i.find('\n') >= 0:
                                    PTCList = PTCList[0].split('\t')
                                    showNp = np.array(PTCList)

                                    PTCList = []
                                    if showNp.shape[0] > 1:  # txt開頭有換行符號則須避開
                                        SingleTxTNp = np.vstack(
                                            (SingleTxTNp, showNp[0:self.Set.imgcol]))  # 資料往下堆疊
                            if i.find('Deg.F') > -1:  # 經過Deg.F 確認開始讀取 PTC
                                StartSaveData = True

                        # 去除SingleTxTNp row=0資料，只留實際PTC資料
                        SingleTxTNp = SingleTxTNp[1:self.Set.imgrow, :]
                        SingleTxTNp_float = SingleTxTNp.astype(
                            np.float)  # convert string array to float

                        AllPTCNp = np.vstack(
                            (AllPTCNp, SingleTxTNp_float))  # 讀完整個txt內容後存入結果
                # 去除AllPTCNp row=0資料，只留實際PTC資料
                ResultNp = AllPTCNp[1:AllPTCNp.shape[0], :].copy()

        # 儲存Label資料
        for i in labellsit:
            i = i.strip(' ')
            PTClabelList.append(self.Set.PTC_class[i])
        PTClabelNp = np.array([PTClabelList])
        PTClabelNp = PTClabelNp.T
        print(ResultNp.shape)

        return ResultNp, PTClabelNp, namelist


資料前處理

In [4]:
import numpy as np

class _preprocess:
    def __init__(self):
        pass

    def splitdata(self, dataNp, ratio, imgrow):
        # input資料row維度
        imgcounter = int(dataNp.shape[0]/imgrow)
        # train維度=all*ratio
        trainCounter = int(imgcounter*ratio)
        trainNp = dataNp[0:trainCounter*imgrow, :]

        # test=all-train
        testNp = dataNp[trainCounter*imgrow:imgcounter*imgrow, :]

        return trainNp, testNp


訓練模型配置

In [9]:
from __future__ import print_function
import sys
from tensorflow import keras
from keras.layers import Dense, Activation, Bidirectional, LSTM, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical
import pickle
import gzip
import urllib.request
import json

def plothistory(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

class CNN_Model:

    def __init__(self):
        pass

    def Classification(self, x_train_all_Np, y_train_all_Np, x_testNp, y_testNp):
        # Assuming _config and _preprocess are defined somewhere in your code
        Set = _config()
        process = _preprocess()

        train_x_np, val_x_np = process.splitdata(x_train_all_Np, 0.8, 1)
        train_y_np, val_y_np = process.splitdata(y_train_all_Np, 0.8, 1)

        img_rows, img_cols = Set.imgrow-1, Set.imgcol
        train_x_np = train_x_np.reshape(int(train_x_np.shape[0]/(img_rows)), img_rows, img_cols, 1)
        val_x_np = val_x_np.reshape(int(val_x_np.shape[0]/(img_rows)), img_rows, img_cols, 1)
        x_train_all_Np = x_train_all_Np.reshape(int(x_train_all_Np.shape[0]/(img_rows)), img_rows, img_cols, 1)

        _input_shape = (img_rows, img_cols, 1)

        x_mean = np.mean(x_train_all_Np, axis=0)
        x_std = np.std(x_train_all_Np, axis=0)

        train_x_np = (train_x_np-x_mean)/x_std
        val_x_np = (val_x_np-x_mean)/x_std

        train_y_np_Onehot = to_categorical(train_y_np, Set.class_num)
        val_y_np_Onehot = to_categorical(val_y_np, Set.class_num)

        model = Sequential()
        model.add(Conv2D(64, kernel_size=(1, 1), activation='relu', input_shape=_input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, kernel_size=(1, 1), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(512, activation='relu'))
        model.add(Dense(1024, activation='relu'))
        model.add(Dense(2048, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(Set.class_num, activation='softmax'))

        model.summary()
        model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

        history = model.fit(train_x_np, train_y_np_Onehot, batch_size=Set.Batch_size, epochs=Set.epoch, validation_data=(val_x_np, val_y_np_Onehot))
        score = model.evaluate(val_x_np, val_y_np_Onehot, verbose=0)

        x_testNp = x_testNp.reshape(int(x_testNp.shape[0]/(img_rows)), img_rows, img_cols, 1)
        x_testNp = (x_testNp-x_mean)/x_std

        predict_x = model.predict(x_testNp)
        predction = np.argmax(predict_x, axis=1)

        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        return predction


class LSTM_Model:
    def __init__(self):
        pass

    def Classification(self, x_train_all_Np, y_train_all_Np, x_testNp, y_testNp):
        Set = _config()
        process = _preprocess()
        # learning_rate = 0.001
        training_iters = Set.epoch
        batch_size = Set.Batch_size
        display_step = 10

        n_input = Set.imgcol
        n_step = Set.imgrow-1
        n_hidden = 256
        n_classes = Set.class_num

        train_x_np, val_x_np = process.splitdata(x_train_all_Np, 0.8, 1)
        train_y_np, val_y_np = process.splitdata(y_train_all_Np, 0.8, 1)

        x_train_all_Np = x_train_all_Np.reshape(-1, n_step, n_input)
        train_x_np = train_x_np.reshape(-1, n_step, n_input)
        val_x_np = val_x_np.reshape(-1, n_step, n_input)

        # train_x_np = train_x_np.astype('float32')
        # val_x_np = val_x_np.astype('float32')

        x_mean = np.mean(x_train_all_Np, axis=0)
        x_std = np.std(x_train_all_Np, axis=0)

        train_x_np = (train_x_np-x_mean)/x_std
        val_x_np = (val_x_np-x_mean)/x_std

        train_y_np_Onehot = keras.utils.to_categorical(train_y_np, n_classes)
        val_y_np_Onehot = keras.utils.to_categorical(val_y_np, n_classes)

        model = Sequential()
        model.add(LSTM(n_hidden, batch_input_shape=(
            None, n_step, n_input), unroll=True))
        # model.add(LSTM(n_hidden,unroll=True))
        # model.add(LSTM(n_hidden))

        # model.add(LSTM(n_hidden,batch_input_shape=(None, n_step, n_input),unroll=True))

        # model.add(Bidirectional(LSTM(units=2,return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(n_classes))
        model.add(Activation('softmax'))

        adam = Adam(lr=0.001)
        model.summary()
        model.compile(optimizer=adam,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        model.fit(train_x_np, train_y_np_Onehot,
                  batch_size=batch_size,
                  epochs=training_iters,
                  verbose=1,
                  validation_data=(val_x_np, val_y_np_Onehot))

        scores = model.evaluate(val_x_np, val_y_np_Onehot, verbose=2)
        print('LSTM test score:', scores[0])
        print('LSTM test accuracy:', scores[1])

        x_testNp = x_testNp.reshape(-1, n_step, n_input)
        x_testNp = (x_testNp-x_mean)/x_std
        predction = model.predict_classes(x_testNp)
        print(predction)
        return predction



主程式

In [13]:
import numpy as np
import tensorflow as tf
from tkinter import font
import os

# Placeholder imports for _file, _config, CNN_Model, and LSTM_Model
#from file import _file
#from config import _config
#from Mymodel import CNN_Model, LSTM_Model

def Project_Classification():
    tXt = _file()
    Set = _config()
    
    # Read train data and test data
    train_x, train_y, trainfilename = tXt.ReadData_TxT(Set.Train_Path)
    test_x, test_y, testfilename = tXt.ReadData_TxT(Set.Test_Path)
    print('=======', train_x.shape, train_y.shape)
    
    Real_y = list()
    for i in testfilename:
        _class = Set.test_y[i]
        Real_y.append(Set.PTC_class[_class])
        
    CNN = CNN_Model()
    Predict_Y = CNN.Classification(train_x, train_y, test_x, test_y)
    
    # Uncomment to use LSTM_Model instead
    # LSTM = LSTM_Model()
    # Predict_Y = LSTM.Classification(train_x, train_y, test_x, test_y)

    Real_yNp = np.array(Real_y)
    print('predict:', Predict_Y)
    print('Real:   ', Real_yNp)
    
    Result = 0
    for i in range(Real_yNp.shape[0]):
        if Predict_Y[i] == Real_yNp[i]:
            Result += 1
    
    Accuracy = (Result / Real_yNp.shape[0]) * 100
    print('Accuracy(%):', Accuracy)

def main():
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use GPU 0
    Project_Classification()

if __name__ == "__main__":
    main()


UnboundLocalError: local variable 'ResultNp' referenced before assignment